In [50]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import segmentation_models as sm
import PIL, cv2

In [2]:
csv_file = pd.read_csv('../../files/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [3]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../files/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../files/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [4]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../files/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../files/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [5]:
csv_file = csv_file[csv_file['fixed_inputs'] != '../../files/train_v2/6384c3e78.jpg']

In [141]:
def get_label_array(mask_paths):
    gt = np.zeros((100, 512, 512, 1))
    for x in range(len(mask_paths)):
        img = cv2.imread(mask_paths[x], cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (512, 512))
        img = img / 255
        img = img.reshape((512, 512, 1))
        img = np.where(img < 0.5, 0, 1)
        gt[x] = img
    return gt

def get_input_array(input_paths):
    gt = np.zeros((100, 512, 512, 3))
    for x in range(len(input_paths)):
        img = PIL.Image.open(input_paths[x])
        img = img.resize((512, 512))
        img = np.array(img)
        img = img / 255
        gt[x] = img
    return gt

In [120]:
gt_mask_paths = csv_file[:100].mask_paths.values
input_paths = csv_file[:100].fixed_inputs.values

In [121]:
gts = get_array(gt_mask_paths)
preds = np.random.random((100, 512, 512, 1))
input_arr = get_input_array(input_paths)

In [92]:
sm_iou_score = sm.metrics.iou_score(gts, preds)

In [83]:
print(sm_iou_score.numpy())

0.0015389751075047446


In [86]:
preds_ = np.where(preds < 0.5, 0, 1)

In [89]:
TP = np.sum(np.logical_and(gts == 1, preds_ == 1))
FP = np.sum(np.logical_and(gts == 0, preds_ == 1))
FN = np.sum(np.logical_and(gts == 1, preds_ == 0))

In [95]:
model = keras.models.load_model('../../unet_incep/', compile=False)

In [133]:
model.save_weights('../../weights.h5')

In [135]:
model = sm.Unet('inceptionv3', classes=1, activation='sigmoid')

87910968/87910968 [==============================] - 0s 0us/step


In [136]:
model.load_weights('../../weights.h5')

In [137]:
model.compile(
    keras.optimizers.SGD(),
    loss = sm.losses.bce_dice_loss,
    metrics = [sm.metrics.iou_score]
)

In [162]:
input_data = tf.data.Dataset.from_tensor_slices((input_arr, gts))
input_data = input_data.batch(10)

input_data_ = tf.data.Dataset.from_tensor_slices((input_arr))
input_data_ = input_data_.batch(10)

In [163]:
model.evaluate(input_data)

10/10 [==============================] - 10s 979ms/step - loss: 0.2918 - iou_score: 0.6458


[0.2918360233306885, 0.6458123922348022]

In [151]:
preds = model.predict(input_data_)

10/10 [==============================] - 10s 969ms/step


In [158]:
last_10 = preds[90:]

In [171]:
gts_splits, pred_splits = np.split(gts, 10), np.split(preds, 10)

In [172]:
scores = []
for x in range(10):
    s = sm.metrics.iou_score(gts_splits[x], pred_splits[x])
    scores.append(s.numpy())

All metrics are summed up and then average taken.

In [181]:
bce = sm.losses.BinaryCELoss()

In [184]:
bce.__call__(gts_splits[0].astype(np.double), pred_splits[0].astype(np.double))

<tf.Tensor: shape=(), dtype=float64, numpy=0.005942681938469484>

In [185]:
sm.losses.bce_dice_loss(gts_splits[0].astype(np.double), pred_splits[0].astype(np.double))

<tf.Tensor: shape=(), dtype=float64, numpy=0.26783273787902184>

In [186]:
losses = []
for x in range(10):
    l = sm.losses.bce_dice_loss(gts_splits[x].astype(np.double), pred_splits[x].astype(np.double))
    losses.append(l.numpy())

In [177]:
np.mean(np.array(scores))

0.6458121528923414

In [188]:
# 0.2918360233306885

In [189]:
np.mean(np.array(losses))

0.2915513158435725